In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
for frame in frames:
    if 'num_participant' not in frame:
        frame['num_participant'] = 10000
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]

data = data[data['num_concurrent'] == 8]
data = data[(data['inner_k'] == 32) & (data['inner_n'] == 80) & (data['outer_k'] == 8) & (data['outer_n'] == 10)]

def protocol_operation(row):
    return f'{row["protocol"]} - {row["operation"]}'

data["protocol - operation"] = data.apply(protocol_operation, axis=1)
data

In [ ]:
hue_order = ["entropy - put", "kademlia - put", "entropy - get", "kademlia - get"]
entropy_palette = sns.cubehelix_palette(n_colors=4)
kademlia_palette = sns.cubehelix_palette(start=2., rot=.4, n_colors=4)
palette = [c for p in zip(entropy_palette, kademlia_palette) for c in p][:4]

sns.relplot(data=data,
    kind="line",
    x="num_participant", y="latency", hue="protocol - operation",
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Number of Nodes") \
.set_ylabels("Latency (s)") \
.savefig("scale.pdf")